# Assignment 1: Transformations


TEAM-ID: 12   
TEAM-NAME: panzer-blow  
YOUR-ID: 2018111024  
YOUR-NAME: Vikrant Dewangan   

(Although you work in groups, both the students have to submit to Moodle, hence there's name field above)

# Instructions
- Please check Moodle for "TEAM-ID" and "TEAM-NAME" fields above. Some of your names have been edited because of redundancy/simplicity. Instructions for submitting the assignment through GitHub Classrooms/Moodle has been uploaded on Moodle. Any clarifications will be made there itself.
- Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment. See `./misc/installation` for detailed step-by-step instructions about the installation setup.
- Both the team members must submit the zip file.
- For this assignment, you will be using Open3D extensively. Refer to [Open3D Documentation](http://www.open3d.org/docs/release/): you can use the in-built methods and **unless explicitly mentioned**, don't need to code from scratch for this assignment. Make sure your code is modular since you will be reusing them for future assignments.
- Take a look at the entire assignment. The descriptive questions at the end might have a clue if you are stuck somewhere.
- Answer the descriptive questions in your own words with context & clarity. Do not just copy-paste from some Wikipedia page. You will be evaluated accordingly.
- Please call the visualization functions only when they are asked. They are asked explicitly at the end of every section.
- You could split the Jupyter Notebook cells where `TODO` is written, but please try to avoid splitting/changing the structure of other cells.

In [1]:
import open3d as o3d
import copy
import numpy as np
from scipy.linalg import logm
from scipy.spatial.transform import Rotation as RR
from scipy.optimize import fsolve
import time

# 1. Getting started with Open3D
## 1.1 Converting RGBD image into Point Cloud


In your robotics journey, it is common to be given just the depth images along with camera parameters in a generic dataset and you'd want to build a 3D data representation out of it, for example, a point cloud. You will understand the math behind these concepts in detail during Vision classes, for now, you can use the in-built functions as a black box.
- Below are the given RGB and D images from SUN RGB-D dataset ([S.Song, CVPR 2015](https://rgbd.cs.princeton.edu/)). 
![low-res-RGBD.resized.png](./misc/low-res-RGBD.png)

- Read these two images `color.jpg` and `depth.png` given in current folder using Open3D. Convert it to a point cloud using the default camera parameters (`o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault`).
Then,
- Create a "world" frame $A$ and combine this (just use $+$ operator) with the above point cloud and save it as `scene.pcd`. Put it aside for now.
- Write a simple function `one_one` to visualize `scene.pcd`.

In [2]:
##############################################################################
# TODO: Do tasks described in 1.1                                            #
##############################################################################

#Read these two images `color.jpg` and `depth.png` given in current folder using Open3D.
color_raw = o3d.io.read_image("color.jpg")
depth_raw = o3d.io.read_image("depth.png")

#Convert it to a point cloud using the default camera parameters
colr=o3d.geometry.Image(color_raw)
dept=o3d.geometry.Image(depth_raw)
rgbd_image = o3d.geometry.RGBDImage.create_from_sun_format(
    color_raw, depth_raw)

# Create a "world" frame  𝐴
A = o3d.geometry.PointCloud()
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(0.4)

# pinhole_camera_intrinsic:
phci = o3d.camera.PinholeCameraIntrinsic(o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault)
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, phci)

A+=pcd
A+=mesh_frame.sample_points_uniformly(40000)

o3d.io.write_point_cloud("scene.pcd", A)

def one_one():
  o3d.visualization.draw_geometries([A])

##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

### Question for 1.1

- In the next code cell, call the function `one_one` here showing `scene.pcd`.

In [3]:
#uncomment the following and add input parameters if any

one_one()

# 2. Rotations, Euler angles and Gimbal Lock

## 2.1 Rotating an object

The objective here is to roughly simulate an object moving on a ground.

- Generate a cube at some point on the ground and create another frame $B$ at the center of this object. Combine these both as a single point cloud `cube.pcd`. (You can pick a point on the ground by using the `get_picked_points` method of the class `open3d.visualization.VisualizerWithEditing`.)
- Now read both the point clouds `scene.pcd` and `cube.pcd` in a script. Whatever tasks you do below are on the object `cube.pcd` (along with the axes $B$) with `scene.pcd` in the background (static).
- Given a sequence of **ZYX Euler** angles $[30^{\circ}, 90^{\circ}, 45^{\circ}]$, generate the rotation. In our case, our object (with its respective axis) undergoes rotation with the background being fixed (with its respective axis).
- Note: Throughout this assignment, we will be using the standard **ZYX** Euler angle convention.
- Write a function `two_one` to show the above by **animation** (cube rotating along each axis one by one).
    - *Hint: Use Open3D's non-blocking visualization and discretize the rotation to simulate the animation. For example, if you want to rotate by $30^{\circ}$ around a particular axis, do in increments of $5^{\circ}$ 6 times to make it look like an animation.*

In [4]:
##############################################################################
# TODO: Do tasks described in 2.1                                            #
##############################################################################
# Replace "pass" statement with your code

# side_length
sl=0.75

mesh_box = o3d.geometry.TriangleMesh.create_box(width=sl,
                                                height=sl,
                                                depth=sl)
mesh_box.paint_uniform_color([1, 0.706, 0])
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(0.75,[sl/2,sl/2,sl/2])
B = mesh_box.sample_points_uniformly(1000)
B+= mesh_frame.sample_points_uniformly(1000)

o3d.io.write_point_cloud("cube.pcd", B)

# o3d.visualization.draw_geometries([cube,scene], zoom=0.3412,
#                                   front=[0.4257, -0.2125, -0.8795],
#                                   lookat=[2.6172, 2.0475, 1.532],
#                                   up=[-0.0694, -0.9768, 0.2024])

# zoom=0.46,front= [ 0.3179877652846847, 0.29517821446711856, -0.90097369708180497 ],up= [ -0.14630023787112281, 0.95419389978225333, 0.26097939002378739 ]
                                

# Now read both the point clouds scene.pcd and cube.pcd in a script.
cube = o3d.io.read_point_cloud("cube.pcd")
scene = o3d.io.read_point_cloud("scene.pcd")

def two_one():
    vis = o3d.visualization.Visualizer()
    vis.create_window()    
    cube_r = copy.deepcopy(cube)
    vis.add_geometry(cube_r)
    vis.add_geometry(scene)
    t=200
    for i in range(t):
#         Rx = cube_r.get_rotation_matrix_from_zyx((0,0,np.pi/(4*t)))  
#         cube_r.rotate(Rx,1)
        Rz = cube_r.get_rotation_matrix_from_zyx((np.pi/(6*t),0,0))
        cube_r.rotate(Rz,1)
        vis.update_geometry(cube_r)
        vis.poll_events()
        vis.update_renderer()
    RR=cube_r.get_rotation_matrix_from_zyx((np.pi/(6),0,0))
    
#     RR=cube_r.get_rotation_matrix_from_zyx((0,0,np.pi/(4)))

    for i in range(t):
        Ry = cube_r.get_rotation_matrix_from_zyx((0,np.pi/(2*t),0))
        cube_r.rotate(np.matmul(np.matmul(RR,Ry),RR.transpose()),1)
        vis.update_geometry(cube_r)
        vis.poll_events()
        vis.update_renderer()
    
    RR=cube_r.get_rotation_matrix_from_zyx((0,np.pi/(2),0))    
    
    for i in range(t):
        Rx = cube_r.get_rotation_matrix_from_zyx((0,0,np.pi/(4*t)))
#         Rz = cube_r.get_rotation_matrix_from_zyx((np.pi/(6*t),0,0))
        cube_r.rotate(np.matmul(np.matmul(RR,Rx),RR.transpose()),1)
        vis.update_geometry(cube_r)
        vis.poll_events()
        vis.update_renderer()
#     '''
#     vis.destroy_window()
    

##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################


### Question for 2.1

- In the next code cell, call the function `two_one` here showing the animation described in section 2.1.

In [5]:
#uncomment the following and add input parameters if any
two_one()


TypeError: rotate(): incompatible function arguments. The following argument types are supported:
    1. (self: open3d.open3d_pybind.geometry.Geometry3D, R: numpy.ndarray[float64[3, 3]], center: numpy.ndarray[float64[3, 1]]) -> open3d.open3d_pybind.geometry.Geometry3D

Invoked with: geometry::PointCloud with 2000 points., array([[ 0.99999657, -0.00261799,  0.        ],
       [ 0.00261799,  0.99999657,  0.        ],
       [ 0.        ,  0.        ,  1.        ]]), 1

## 2.2 Euler angle & Gimbal lock

Code the following yourself from scratch (Refer Craig book - Section: $Z-Y-X$ Euler angles - same conventions/notations followed).

- Case 1: Given the rotation matrix $M_{given}$ below, extract Euler angles $\alpha , \beta ,\gamma$. Convert it back to the rotation matrix $M_{recovered}$ from Euler angles.

    $$M(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0.26200263 & -0.19674724 & 0.944799 \\0.21984631 & 0.96542533 & 0.14007684 \\
    -0.93969262 & 0.17101007 & 0.29619813\end{array}\right] $$

    After coding it from scratch, check your calculations using `scipy.spatial.transform.Rotation`. (Mandatory)

- Case 2: Given the rotation matrix $N_{given}$, extract Euler angles, and convert back $N_{recovered}$.

    $$N(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0 & -0.173648178 &  0.984807753 \\0 & 0.984807753 & 0.173648178 \\
    -1 & 0 & 0\end{array}\right] $$

    Again use `scipy` and check its output. If `scipy` is showing any warnings on any of the above cases, explain it in "**Questions for 2.2**" (last question). Write code in the next cell.
    
- (Optional) Case 3: Do the above two for quaternion using scipy functions, i.e. given the rotation matrix, extract quaternion and convert back.

In [6]:
##############################################################################
# DON'T EDIT
M_given =  np.array([[0.26200263, -0.19674724, 0.944799],
                     [0.21984631, 0.96542533, 0.14007684],
                     [-0.93969262, 0.17101007, 0.29619813]])

N_given = np.array([[0,-0.173648178,0.984807753],
                    [0, 0.984807753, 0.173648178],
                    [-1, 0, 0]])

In [7]:
# TODO: Do tasks described in 2.2                                            #
##############################################################################
# Replace "pass" statement with your code

def eulerToRotation(a,b,g):
        sa=np.sin(a)
        sb=np.sin(b)
        sg=np.sin(g)
        ca=np.cos(a)
        cb=np.cos(b)
        cg=np.cos(g)
        return np.array([[ca*cb ,ca*sb*sg-sa*cg ,ca*sb*cg+sa*sg],
                         [sa*cb ,sa*sb*sg+ca*cg ,sa*sb*cg-ca*sg ],
                         [-sb   ,cb*sg          ,cb*cg ]])


for case in [1,2]:
    print("Case : "+str(case))
    if case==1:
        R=M_given.copy()
    elif case ==2:
        R=N_given.copy()


    b=np.arcsin(-R[2][0])

    if abs(np.cos(b))>10e-6:
        a=np.arctan2(R[1][0]/np.cos(b),R[0][0]/np.cos(b))
        g=np.arctan2(R[2][1]/np.cos(b),R[2][2]/np.cos(b))
    else:
        a=0
        g=np.arctan2(-R[2][0]*R[0][1],-R[2][0]*R[0][2])

    print(a,b,g)

    
    #SciPy
    r=RR.from_euler('ZYX',[a,b,g])

    if case==1:
        my_array_case1=[a,b,g]
        M_recovered=r.as_matrix()
    elif case==2:
        my_array_case2=[a,b,g]
        N_recovered=r.as_matrix()


    print("Manual Check : ")
    print(eulerToRotation(a,b,g))
    print("SciPy Check :")
    print(r.as_matrix())
    print("Given Matrix :")
    print(R)
    print()
    print()
    print()

##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

Case : 1
0.6981317003487099 1.2217304740981882 0.5235987753730347
Manual Check : 
[[ 0.26200263 -0.19674724  0.944799  ]
 [ 0.21984631  0.96542533  0.14007684]
 [-0.93969262  0.17101007  0.29619813]]
SciPy Check :
[[ 0.26200263 -0.19674724  0.944799  ]
 [ 0.21984631  0.96542533  0.14007684]
 [-0.93969262  0.17101007  0.29619813]]
Given Matrix :
[[ 0.26200263 -0.19674724  0.944799  ]
 [ 0.21984631  0.96542533  0.14007684]
 [-0.93969262  0.17101007  0.29619813]]



Case : 2
0 1.5707963267948966 -0.17453292552956243
Manual Check : 
[[ 6.12323400e-17 -1.73648178e-01  9.84807753e-01]
 [ 0.00000000e+00  9.84807753e-01  1.73648178e-01]
 [-1.00000000e+00 -1.06328843e-17  6.03020831e-17]]
SciPy Check :
[[ 0.00000000e+00 -1.73648178e-01  9.84807753e-01]
 [ 0.00000000e+00  9.84807753e-01  1.73648178e-01]
 [-1.00000000e+00  0.00000000e+00 -5.55111512e-17]]
Given Matrix :
[[ 0.         -0.17364818  0.98480775]
 [ 0.          0.98480775  0.17364818]
 [-1.          0.          0.        ]]





## Questions for 2.2
- Have you used `np.arctan` or an any equivalent `atan` function above? Why or why not?   
    * Ans: *Yes we have used `np.arctan2` function to get value of `alpha` and `gama`*

### For Case 1 above,
- What Euler angles  $\alpha , \beta ,\gamma$ did you get? Replace `my_array_case1` with your array.

In [8]:
# Uncomment and replace my_array_case1 with your array.
print("My Euler angles for case 1 are" + str(my_array_case1))

My Euler angles for case 1 are[0.6981317003487099, 1.2217304740981882, 0.5235987753730347]


  - Were you able to recover back your rotation matrix when you converted it from Euler angles? Why/why not? Replace `M_given` and `M_recovered` with your matrices below and explain "why/why not" after this code snippet.

In [9]:
# Uncomment and Replace M_given and M_recovered with your matrices below.
error = np.linalg.norm(logm(M_given @ M_recovered.T))
print("For case 1, it is " + str(error<0.0001) + " I could recover the original matrix.")

For case 1, it is True I could recover the original matrix.


- Why/why not? Based on your observations here, is there any problem with Euler angle representation for Case 1? If yes, what is it?

    - Ans:    *No Problem occured, because cos(beta)!=0 hence the ratio of R[2][1] and R[2][2] (and equivalent) will be defined.*

### Repeat the above for Case 2.

In [10]:
# Uncomment and Replace N_given and N_recovered with your matrices below.
print("My Euler angles for case 2 are" + str(my_array_case2))
error = np.linalg.norm(logm(N_given @ N_recovered.T))
print("For case 2, it is " + str(error<0.0001) + " I could recover the original matrix.")

My Euler angles for case 2 are[0, 1.5707963267948966, -0.17453292552956243]
For case 2, it is True I could recover the original matrix.


* Why/why not? Based on your observations here, is there any problem with Euler angle representation for Case 2? If yes, what is it?

    * Ans: No & Yes.One problem is that any value of alpha will work for case 2(We have specificly taken it as 0). Second problem could have arrised if we have gone with simple/normal methord of calcualting out as both R[2][1] and R[2][2] would have been 0 and there ratio would have not been defined. We have taken care of this by using other available information in Given Matrix*
    
    
* Explain any more problems with Euler angle representation. Explain what you understand by Gimbal lock (concisely in your own words). You could revisit this question in the section 2.4.
    * Ans: *Another issue with euler angles is Gimble Lock. In some cases axis align in such a way that rotation about all axis causes the same effect and you cant come out of the state .*

- When you used `scipy.spatial.transform.Rotation` for the above 2 cases,
    - Have you used `zyx` above in `r.as_euler('')` argument? Why or why not? Explain the difference between extrinsic and instrinsic rotations with equivalent technical names from Craig book?
        * Ans: *No We have used ZYX in argument as this corresponds to Intrensic Rotation where as zyx are for extrensic rotation.Now w.r.t Craig book Z-Y-X Fix Angles matches with Extrensic rotation where rotations are made wrt fixed axises where as in Z-Y-X Euler Angles rotations are done wrt axis of body (which changes after every rotation) and hence called intrensic rotation *
    - Has `scipy` shown any warnings on any of the above cases? If yes, explain it.
        * Ans: *Not showing :/*
    - (Optional) For Case 3 above (quaternion) which you did using scipy, did you observe any problem with quaternion? Depending on your observations, which is better? Quaternion or Euler angles? And why?
         * Ans: *your answer here.*


## 2.3 Rotation matrix as an Operator
This question will help you in your understanding of [Rotator-transform (Vector-frame) equivalence](https://www.notion.so/saishubodh/Lecture-2-Transformations-11d69d8cef2d4cd195a98fa7d33224e1#f90ece4f5e374743bfed47e46a83ecfe).

![image.png](./misc/xyz-frame.png)
Consider the frame $XYZ$ in the above image. Say you have a vector $x_1=[0,\sqrt{3},0]$. Now you want to rotate it such that you end up at $x_2=[1,1,1]$ through a sequence of Euler angle rotations. Your goal is to find out those $\alpha, \beta \: \& \: \gamma$ ($ZYX$). We will follow this order whenever we refer to it below.

First, properly understand the so-called "Rotator-transform equivalence" to figure out what are the terms of rotation matrix. Then, put the math on paper and you will end up with a set of non-linear equations. Write the set of linear equations in LaTeX here:

$$ \boldsymbol{\mathbf{x_{2}}}= \boldsymbol{\mathbf{R}}\boldsymbol{\mathbf{x_{1}}}\\\\
\boldsymbol{\mathbf{ R}}= \left[\begin{array}{ccc}c \alpha c \beta & c \alpha s \beta s \gamma-s \alpha c \gamma & c \alpha s \beta c \gamma+s \alpha s \gamma \\s \alpha c \beta & s \alpha s \beta s \gamma+c \alpha c \gamma & s \alpha s \beta c \gamma-c \alpha s \gamma \\-s \beta & c \beta s \gamma & c \beta c \gamma\end{array}\right]\\\\
\begin{bmatrix}
1\\ 
1\\ 
1
\end{bmatrix}=\boldsymbol{\mathbf{R}}\begin{bmatrix}
0\\ 
\sqrt{3}\\ 
0
\end{bmatrix}
 $$

Solve these equations using `fsolve` from `scipy.optimize` as follows: (Come back and answer the following questions after coding it in the next block)
- `case1`: First, solve it with an initialization of (0,0,0). Check if your answer is correct using `np.isclose`.
    * What Euler angles did you get? Answer in $\alpha, \beta \: \& \: \gamma$ format:
        * Ans: *$a_1, b_1, c_1$ = -0.32906484, -93.69048524,   0.74815098* 
- `case2`: Now, forget about the solver for a moment: Can you visualize and think of sequence of rotations one by one to reach the final position (which is different than previous set of rotations)? Now, validate your answer by giving (your answer $\pm 5$) as initialization.
    * What Euler angles did you get? Answer in $\alpha, \beta \: \& \: \gamma$ format:
        * Ans: *$a_2, b_2, c_2$ = -0.785398178e, -0.0000000205, 0.615479709*

In [11]:
##############################################################################
# DON'T EDIT
x_1 = np.array([0,np.sqrt(3),0])
x_2 = np.array([1,1,1])

In [12]:
##############################################################################
# TODO: Do tasks described in 2.3                                            #
##############################################################################

def func(x):
    sa=np.sin(x[0])
    sb=np.sin(x[1])
    sg=np.sin(x[2])
    ca=np.cos(x[0])
    cb=np.cos(x[1])
    cg=np.cos(x[2])
    Rm= np.array([[ca*cb ,ca*sb*sg-sa*cg ,ca*sb*cg+sa*sg],
                     [sa*cb ,sa*sb*sg+ca*cg ,sa*sb*cg-ca*sg ],
                     [-sb   ,cb*sg          ,cb*cg ]])
    return np.matmul(Rm,x_1)-x_2

for case in [1,2]:
    if case==1:
        print("Case 1 :")
        root = fsolve(func, [0, 0, 0])
        print("Root : "+str(root))
        print("Check : ",end='')
        print(np.isclose(func(root), [0.0, 0.0,0.0]))
        print()
        a1,b1,c1=root
    if case==2:
        print("Case 2 :")
        root = fsolve(func, [-np.pi/4, 0, np.arcsin(1/np.sqrt(3))])
        print("Root : "+str(root))
        print("Check : ",end='')
        print(np.isclose(func(root), [0.0, 0.0,0.0]))
        print()
        a2,b2,c2=root
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

Case 1 :
Root : [ -0.32906485 -93.69048525   0.74815097]
Check : [ True  True  True]

Case 2 :
Root : [-0.78539816  0.          0.61547971]
Check : [ True  True  True]



In [13]:
# From Section 2.2, use the function which takes Euler angles and gives Rotation matrix as output.
# Uncomment and replace `r_mat` with the name of the function. (Do NOT edit anything else)

x_2_obtained_case1 = eulerToRotation(a1,b1,c1) @ x_1.T 
x_2_obtained_case2 = eulerToRotation(a2,b2,c2) @ x_1.T 
test = True 

In [14]:
# DON'T EDIT
if test == True:
    case1_test = np.isclose(x_2_obtained_case1,  np.array([1.0,1.0,1.0]))
    case2_test = np.isclose(x_2_obtained_case2,  np.array([1.0,1.0,1.0]))
    print("For case 1, it is " + str(bool(case1_test[0] and case1_test[1] and case1_test[2])) + 
          " that I could end up at (1,1,1) after rotation.")
    print("For case 2, it is " + str(bool(case2_test[0] and case2_test[1] and case2_test[2])) + 
          " that I could end up at (1,1,1) after rotation.")

For case 1, it is True that I could end up at (1,1,1) after rotation.
For case 2, it is True that I could end up at (1,1,1) after rotation.


## 2.4 Gimbal Lock visualization (Optional)

A nice visualization video for gimbal lock is [this](https://www.youtube.com/watch?v=zc8b2Jo7mno). You are about to animate a similar visualization demonstrating gimbal lock 😃.

![image.png](./misc/xyz-frame.png)

- Write a function `two_four` for the visualization of gimbal lock. Follow the below steps to get the intuition going. Use Open3D for the following.
    - Say our frame's initial position is as the above image. Now the final goal at the end of rotation is to get the $Y$ axis pointing in the direction of the vector $(x,y,z)$ that you currently see in the above image. This point is fixed in space and is NOT moving as we rotate our axis.
    - For creating that point, you could use a small sphere using `open3d.geometry.create_mesh_sphere`. You already know how to create an axis by now.
    - Following our $ZYX$ convention, first rotate your frame about $Z$ axis by an angle, say $-35^{\circ}$. Then rotate about $Y$ axis by an angle ${\beta}$ and then about $X$ by say $55^{\circ}$.
        - Are there any specific angle(s) $\beta$ using which you will **never** reach our point $(x,y,z)$ ?
            - Clue: We are specifically talking about gimbal lock here and notice the word "never".
        - Under this (these) specific angle(s) of $\beta$ & different combinations of $\alpha$ and $\gamma$, make an animation and clearly show why your $Y$ axis is unable to align in the direction of that vector $(x,y,z)$ using the animation.

            If you are unsure to simulate the animation, you could do it as follows:

            - You could first fix some $\alpha$, say $-35^{\circ}$ & an above value of $\beta$, you can now vary $\gamma$ from $-180^{\circ} \text{ to }180^{\circ}$ to simulate the animation.
            - Now fix another $\alpha$, say $45^{\circ}$ and repeat the above process. So that's 2 specific values of $\alpha$.
            - Show this for all angles of $\beta$ if there are more than 1.
            - Therefore, when the code is run, there should be a minimum of (2 $\times$ (number of values of $\beta$)) animations. 2 for values of $\alpha$, you could show it for even more if you wish to.

In [17]:
##############################################################################
# TODO: Do tasks described in 2.4                                            #
##############################################################################
# Replace "pass" statement with your code
def two_four():
    # alpha = -35
    alpha_angle = -35
#   beta = 90
    lock_frame = open3d.geometry.TriangleMesh.create_coordinate_frame(0.4)
    beta_angle = 90
    gamma_angle = -180
    vis = o3d.visualization.Visualizer()
    vis.create_window()    
    vis.add_geometry(lock_frame)

    #about Z
    Rz = lock_frame.get_rotation_matrix_from_zyx((np.deg2rad(alpha_angle),0,0))
    lock_frame.rotate(Rz,1)   
    vis.update_geometry(lock_frame)
    vis.poll_events()
    vis.update_renderer()

    #about Y
    RR=lock_frame.get_rotation_matrix_from_zyx((np.deg2rad(alpha_angle),0,0))
    Ry = lock_frame.get_rotation_matrix_from_zyx((0,np.deg2rad(beta_angle),0))
    lock_frame.rotate(np.matmul(np.matmul(RR,Ry),RR.transpose()),1)
    vis.update_geometry(lock_frame)
    vis.poll_events()
    vis.update_renderer()

    #about X
    RR=lock_frame.get_rotation_matrix_from_zyx((0,np.deg2rad(beta_angle),0))
    for gamma_angle in range(-180,181,1):    
        Rz = lock_frame.get_rotation_matrix_from_zyx((0,0,np.deg2rad(gamma_angle)))
        lock_frame.rotate(np.matmul(np.matmul(RR,Rz),RR.transpose()),1)    
        vis.update_geometry(lock_frame)
        vis.poll_events()
        vis.update_renderer()
    

#   beta = -90    
    beta_angle = -90
    print("DONE\n")
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

> VOILA! You have just animated the famous Gimbal lock problem. If you are curious, read about the [Apollo 11](https://en.wikipedia.org/wiki/Gimbal_lock#On_Apollo_11) Gimbal lock incident.

### Questions for 2.4 (Optional)

- Mention the value(s) of $\beta$ here: 
    * Ans: +90, -90.
- Now that you understand gimbal lock through visualization, explain it now in matrix form: For the above values of $\beta$, what does the rotation matrix look like? Can you explain why gimbal lock occurs from the rotation matrix alone? Clue: Use sin/cos formulae. 
    * Ans: $$ \begin{bmatrix} 0 & cos (\alpha) sin (\gamma) - cos (\gamma) sin (\alpha) & cos (\alpha) cos (\gamma) - sin (\gamma)  sin (\alpha) \\  
    0 & cos (\alpha) cos (\gamma) - sin (\gamma)  sin (\alpha) &  - cos (\alpha) sin (\gamma) + cos (\gamma) sin (\alpha) \\
    -1 & 0 & 0
    \end{bmatrix} $$
    In the matrix, and for any value of $\alpha$ or $\gamma$, it will always rotate about X axis only.  
- Call the function `two_four` for the visualization of gimbal lock written above. 

In [18]:
#uncomment the following and add input parameters if any
two_four()